# Import

In [1]:
import csv
import os

import numpy as np
import pandas as pd
from tqdm import tqdm
from gensim.models.doc2vec import Doc2Vec, TaggedDocument

<jemalloc>: MADV_DONTNEED does not work (memset will be used instead)
<jemalloc>: (This is the expected behaviour if you are running under QEMU)


In [2]:
tqdm.pandas()

# Read data

In [3]:
ag_news_df = pd.read_csv("../../Preprocessing/data/AgNews/master.csv", index_col=0)

In [4]:
with open("../../Preprocessing/data/AgNews/class.csv", mode="r") as f:
    reader = csv.reader(f)
    class_labels = [label for label in reader]

In [5]:
documents = [
    TaggedDocument(doc.split(" "), [i]) for i, doc in enumerate(ag_news_df.words)
]

In [6]:
vector_sizes = [2, 3, 4, 6, 7, 8, 9, 10, 20, 40, 80, 160, 320, 640]

In [7]:
def get_doc2vec(texts, vector_size,seed, path):
    model = Doc2Vec(
        documents,
        vector_size=vector_size,
        window=5,
        min_count=1,
        epochs=50,
        seed=seed,
        workers=os.cpu_count(),
    )
    model.save(path)
    vectors = [model.infer_vector(text) for text in texts]
    return vectors

In [8]:
model_nums = 1

In [9]:
vectors_path = "../data/AgNews/doc2vec/vector/"
models_path = "../data/AgNews/doc2vec/model/"
for vector_size in tqdm(vector_sizes):
    for model_num in range(model_nums):
        vector_path = f"{vectors_path}/{vector_size}/"
        model_path = f"{models_path}/{vector_size}/"
        os.makedirs(vector_path, exist_ok=True)
        os.makedirs(model_path, exist_ok=True)

        vectors = get_doc2vec(
            ag_news_df.words.apply(lambda word: word.split(" ")),
            vector_size,
            seed=model_num,
            path=f"{model_path}{model_num}.model",
        )

        np.savetxt(
            f"{vector_path}{model_num}.csv",
            np.stack(vectors),
            delimiter=",",
        )

100%|██████████| 14/14 [25:47:51<00:00, 6633.68s/it]  
